In [35]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)

In [36]:
# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

# Data Cleaning

Even though we are interested only in restaurants, let's give a look at the categories of the businesses in the dataset.

In [37]:
# 'Category' may contain multiple values (e.g., ['Pharmacy', 'Health']), let's use aggregation to extract all unique values across documents

categories = meta.aggregate([
    {"$unwind": {"path": "$category"}}, 
    {"$group": {"_id": "$category"}},
    {"$sort": {"_id": 1}}  # Sort alphabetically
])

# Convert to list
categories = [doc["_id"] for doc in categories]
print(categories)

['ATM', 'ATV dealer', 'ATV rental service', 'ATV repair shop', 'Aboriginal and Torres Strait Islander organisation', 'Aboriginal art gallery', 'Abortion clinic', 'Abrasives supplier', 'Accountant', 'Accounting firm', 'Acupuncture clinic', 'Acupuncturist', 'Acura dealer', 'Addiction treatment center', 'Adoption agency', 'Adult DVD store', 'Adult day care center', 'Adult education school', 'Adult entertainment club', 'Adult entertainment store', 'Adventure sports', 'Adventure sports center', 'Advertising agency', 'Advertising service', 'Aerial photographer', 'Aerial sports center', 'Aerospace company', 'After school program', 'Aged care', 'Aggregate supplier', 'Agricultural cooperative', 'Agricultural organization', 'Agricultural product wholesaler', 'Agricultural production', 'Agricultural service', 'Aikido club', 'Air compressor repair service', 'Air compressor supplier', 'Air conditioning contractor', 'Air conditioning repair service', 'Air conditioning store', 'Air conditioning syste

Let's delete all the businesses that do not have 'restaurant' in the 'category' field.

In [38]:
# Step 1: Get the IDs of matching documents
matching_ids = [
    doc["_id"] for doc in meta.find(
        {"category": {"$elemMatch": {"$regex": "restaurant", 
                                     "$options": "i"}}, 
         "state": {"$ne": "Permanently closed"}}, 
        {"_id": 1}  # Fetch only the _id field for efficiency
    )
]

# Step 2: Count total documents before deletion
total_docs_before = meta.count_documents({})

# Step 3: Delete documents that are NOT in the matching list
if matching_ids:  
    result = meta.delete_many({"_id": {"$nin": matching_ids}})
    # Get count of deleted documents
    total_deleted = result.deleted_count  
    # Count remaining documents
    total_docs_after = meta.count_documents({})  
    
    print(f"Deleted {total_deleted} documents that did not match the query.")
    print(f"Total documents before: {total_docs_before}, after: {total_docs_after}")
else:
    print("No matching documents found.")


Deleted 17782 documents that did not match the query.
Total documents before: 21507, after: 3725


Let's remove those attributes that are not interesting for our purposes: 'relative_results', 'url'

In [39]:
meta.update_many({}, {"$unset": {"relative_results":"", "url": ""}})

UpdateResult({'n': 3725, 'nModified': 3725, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

# Create Indexes

Let's create indexes on our data collections for query efficiancy purposes.

Indexe for reviews:

In [40]:
# Create index on reviews
reviews.create_index(
    {'rating':-1,'num_of_reviews':-1}
)

'rating_-1_num_of_reviews_-1'

### Indexes on restaurants' meta data:

In [41]:
# Create index on average rating
meta.create_index(
    {'name':1, 'avg_rating': -1}
)

'name_1_avg_rating_-1'

We want to create an index based on restaurants' position to enable queries to filter them based on such data.

In [42]:
# Update the schema to store the location as GeoJSON
meta.update_many(
    {},
    [
        { 
            "$set": {
                "location": {
                    "type": "Point",
                    "coordinates": [
                        { "$toDouble": "$longitude" },
                        { "$toDouble": "$latitude" }
                    ]
                }
            }
        }
    ]
)

UpdateResult({'n': 3725, 'nModified': 3725, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Once the location field is set up, create the 2dsphere index

In [43]:
meta.create_index([("location", "2dsphere")])

'location_2dsphere'

In [44]:
# Check whether the 2dsphere index has been created correctly
print(meta.index_information())


{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'name_1_avg_rating_-1': {'v': 2, 'key': [('name', 1), ('avg_rating', -1)]}, 'location_2dsphere': {'v': 2, 'key': [('location', '2dsphere')], '2dsphereIndexVersion': 3}}


In [45]:
# Find documents that have the 'location' field
documents_with_location = meta.find({"location": {"$exists": True}})

# Print the first 1 documents with 'location' to check the data
for doc in documents_with_location.limit(1):
    print(doc)

{'_id': ObjectId('67c9a9928c4ae1c3ac1c76e1'), 'name': 'Hale Pops', 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762', 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33', 'description': None, 'latitude': 21.637795699999998, 'longitude': -157.9207142, 'category': ['Restaurant'], 'avg_rating': 4.4, 'num_of_reviews': 18, 'price': None, 'hours': [['Thursday', '11AM–8PM'], ['Friday', '11AM–8PM'], ['Saturday', '11AM–8PM'], ['Sunday', 'Closed'], ['Monday', '11AM–8PM'], ['Tuesday', '11AM–8PM'], ['Wednesday', '11AM–8PM']], 'MISC': {'Service options': ['Outdoor seating', 'Takeout', 'Delivery'], 'Popular for': ['Lunch', 'Solo dining'], 'Accessibility': ['Wheelchair accessible entrance'], 'Offerings': ['Comfort food', 'Quick bite'], 'Amenities': ['Good for kids'], 'Atmosphere': ['Casual'], 'Crowd': ['Groups', 'Tourists'], 'Payments': ['NFC mobile payments']}, 'state': 'Closed ⋅ Opens 11AM', 'location': {'type': 'Point', 'coordinates': [-157.9207142, 21.637795699999998]}}


# Convert hours <br>
Convert opening and closing times

The following is a document of the collection MetaData (the one containing the data of businesses). We want to convert the format of 'hour' 
```python
# Example data
restaurant_data = {
    "name": "Kraken Coffee Kahului",
    "category": "Restaurant",
    "hours": [['Thursday', '11AM–8PM'], ['Friday', '11AM–8PM'], 
    ['Saturday', '11AM–8PM'], ['Sunday', 'Closed'], ['Monday', '11AM–8PM'], 
    ['Tuesday', '11AM–8PM'], ['Wednesday', '11AM–8PM']],
    "location": {"type": "Point", "coordinates": [-156.4506136, 20.8882377]}
}
```

The purpose is to have a more structured format for 'hour', we want each document to explicit its opening and closing time for each day and obtain in the end something like this:

[{'day': 'Thursday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Friday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Saturday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Sunday', 'open_time': '99:99', 'close_time': '99:99'}, {'day': 'Monday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Tuesday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Wednesday', 'open_time': '11:00', 'close_time': '20:00'}]


In [46]:
from datetime import datetime

def convert_hours(hours_list):
    if not hours_list or not isinstance(hours_list, list):
        return []  # Return empty list if invalid

    formatted_hours = []
    
    for entry in hours_list:
        if isinstance(entry, dict):  
            # Already formatted, keep as is
            formatted_hours.append(entry)  
            continue  

        if not isinstance(entry, list) or len(entry) != 2:
            print(f"Skipping invalid entry: {entry}")  
            continue  

        day, time_range = entry

        if not isinstance(day, str) or not \
            isinstance(time_range, str):
            print(f"Skipping invalid data format: {entry}")  
            continue  

        # Handle 'Closed' case 
        # -> Assign '99:99' to indicate closure
        if time_range.lower() == "closed":
            formatted_hours.append({
                "day": day,
                "open_time": "99:99",
                "close_time": "99:99"
            })
            continue  

        # Handle 'Open 24 hours' case
        if "24 hours" in time_range.lower():
            formatted_hours.append({
                "day": day,
                "open_time": "00:00",
                "close_time": "23:59"
            })
            continue  

        try:
            # Split by en-dash
            open_time, close_time = time_range.split("–")  
        except ValueError:
            print(f"Skipping invalid time format: {entry}")  
            continue  

        # Ensure open_time gets the same AM/PM 
        # indication as close_time if missing
        if "AM" in close_time or "PM" in close_time:
            if not ("AM" in open_time or "PM" in open_time):
                open_time += "AM" \
                    if "AM" in close_time else "PM"

        # Convert open time
        try:
            if ":" in open_time:
                open_time = datetime.strptime(
                    open_time.strip(), "%I:%M%p").strftime(
                        "%H:%M")
            else:
                open_time = datetime.strptime(
                    open_time.strip(), "%I%p").strftime(
                        "%H:%M")
        except ValueError:
            print(f"Skipping invalid open_time: \
                  {open_time} (close_time is {close_time})")  
            continue  

        # Convert close time
        try:
            if ":" in close_time:
                close_time = datetime.strptime(
                    close_time.strip(), "%I:%M%p").strftime(
                        "%H:%M")
            else:
                close_time = datetime.strptime(
                    close_time.strip(), "%I%p").strftime(
                        "%H:%M")
        except ValueError:
            print(f"Skipping invalid close_time: \
                  {close_time} (open_time is {open_time})")  
            continue  

        formatted_hours.append({
            "day": day,
            "open_time": open_time,
            "close_time": close_time
        })

    return formatted_hours





# Example data with 'hours' in a format like the raw one, let's see the output we get after the conversion
restaurant_data = {
    "name": "Kraken Coffee Kahului",
    "category": "Restaurant",
    "hours": [['Thursday', '11AM–8PM'], 
              ['Friday', '11AM–8PM'], 
              ['Saturday', '11AM–8PM'], 
              ['Sunday', 'Closed'], 
              ['Monday', '11AM–8PM'], 
              ['Tuesday', '11AM–8PM'], 
              ['Wednesday', '11AM–8PM']],
    "location": {"type": "Point", "coordinates": 
                 [-156.4506136, 20.8882377]}
}

# Apply the function
formatted_hours = convert_hours(restaurant_data["hours"])

# Print the output
print(formatted_hours)



[{'day': 'Thursday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Friday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Saturday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Sunday', 'open_time': '99:99', 'close_time': '99:99'}, {'day': 'Monday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Tuesday', 'open_time': '11:00', 'close_time': '20:00'}, {'day': 'Wednesday', 'open_time': '11:00', 'close_time': '20:00'}]


Let's convert the format of hours for all documents

In [47]:
# Initialize a counter for documents not updated
not_updated_count = 0

# Process all documents with the "hours" field
for doc in meta.find({"hours": {"$exists": True}}):
    try:
        # Transform hours
        updated_hours = convert_hours(doc["hours"])  
        
        if updated_hours:
            # Only update if we have valid hours  
            meta.update_one({"_id": doc["_id"]}, 
                            {"$set": {"hours": updated_hours}
                             })
            #print(f"Updated document ID: {doc['_id']}")
        else:
            # Increment the counter if hours 
            # are empty or invalid
            not_updated_count += 1
            print(f"Skipping update for document ID \
                  {doc['_id']} due to empty or \
                    invalid hours.")

    except Exception as e:
        print(f"Error processing document ID\
               {doc['_id']}: {e}")
        # Increment count on exception
        not_updated_count += 1  

print(f"All documents processed. {not_updated_count}\
       documents were not updated.")


Skipping update for document ID                   67c9a9928c4ae1c3ac1c7719 due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c771b due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c776f due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c7771 due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c785b due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c790d due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c7a38 due to empty or                     invalid hours.
Skipping update for document ID                   67c9a9928c4ae1c3ac1c7a3e due to empty or                     invalid

Show a restaurant with the new **'hour'** format

In [48]:
example = meta.find_one({})

# Print the document if found
if example:
    from pprint import pprint  # Pretty print for better readability
    pprint(example)


{'MISC': {'Accessibility': ['Wheelchair accessible entrance'],
          'Amenities': ['Good for kids'],
          'Atmosphere': ['Casual'],
          'Crowd': ['Groups', 'Tourists'],
          'Offerings': ['Comfort food', 'Quick bite'],
          'Payments': ['NFC mobile payments'],
          'Popular for': ['Lunch', 'Solo dining'],
          'Service options': ['Outdoor seating', 'Takeout', 'Delivery']},
 '_id': ObjectId('67c9a9928c4ae1c3ac1c76e1'),
 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762',
 'avg_rating': 4.4,
 'category': ['Restaurant'],
 'description': None,
 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33',
 'hours': [{'close_time': '20:00', 'day': 'Thursday', 'open_time': '11:00'},
           {'close_time': '20:00', 'day': 'Friday', 'open_time': '11:00'},
           {'close_time': '20:00', 'day': 'Saturday', 'open_time': '11:00'},
           {'close_time': '99:99', 'day': 'Sunday', 'open_time': '99:99'},
           {'close_time': '20:00', 'day': 'Monday', 

In order to ease the filtering of restaurants based on time (hour of the day) let's convert all the opening hours in seconds from midnight.

In [49]:
def time_to_seconds(time_str):
    # If it's already an integer, return as is
    if isinstance(time_str, int):  
        return time_str
    
    # Handle unexpected types
    if not isinstance(time_str, str):  
        return None  

    # Split the time string by ":"
    time_parts = time_str.split(":")
    
    # Check if the string is correctly 
    # formatted (should have exactly two parts)
    if len(time_parts) == 2:
        try:
            # Convert hours and minutes to integers
            hours = int(time_parts[0])
            minutes = int(time_parts[1])
            
            # Calculate total seconds from midnight
            return hours * 3600 + minutes * 60
        except ValueError:
            # Handle case where hours or 
            # minutes are not integers
            return None  
    else:
        # Handle invalid time format (not "HH:MM")
        return None  
    

print(time_to_seconds("23:30"))  # Should print 84600 
print(time_to_seconds("10:45"))  # Should print 38700 
print(time_to_seconds("14:47"))  # Should print 53220


84600
38700
53220


Loop for converting opening and closing time of all restaurants:

In [50]:
# Find all documents
for doc in meta.find({}):
    # Get the 'hours' field safely  
    hours_list = doc.get("hours")

    # Skip if 'hours' is missing or not a list
    if not isinstance(hours_list, list):  
        continue

    updated_hours = []

    for entry in hours_list:
        # Ensure each entry is a dictionary  
        if not isinstance(entry, dict):  
            continue

        updated_hours.append({
            "day": entry.get("day", ""),  
            "open_time": time_to_seconds(
                entry.get("open_time", "99:99")),  
            "close_time": time_to_seconds(
                entry.get("close_time", "99:99"))  
        })

    if updated_hours:
        meta.update_one({"_id": doc["_id"]}, 
                        {"$set": {"hours": updated_hours}})

print("All documents updated successfully!")


All documents updated successfully!


In [51]:
example = meta.find_one({})

# Print the document if found
if example:
    from pprint import pprint  # Pretty print for better readability
    pprint(example)


{'MISC': {'Accessibility': ['Wheelchair accessible entrance'],
          'Amenities': ['Good for kids'],
          'Atmosphere': ['Casual'],
          'Crowd': ['Groups', 'Tourists'],
          'Offerings': ['Comfort food', 'Quick bite'],
          'Payments': ['NFC mobile payments'],
          'Popular for': ['Lunch', 'Solo dining'],
          'Service options': ['Outdoor seating', 'Takeout', 'Delivery']},
 '_id': ObjectId('67c9a9928c4ae1c3ac1c76e1'),
 'address': 'Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762',
 'avg_rating': 4.4,
 'category': ['Restaurant'],
 'description': None,
 'gmap_id': '0x7c00456eecad3111:0x8217f9600c51f33',
 'hours': [{'close_time': 72000, 'day': 'Thursday', 'open_time': 39600},
           {'close_time': 72000, 'day': 'Friday', 'open_time': 39600},
           {'close_time': 72000, 'day': 'Saturday', 'open_time': 39600},
           {'close_time': 362340, 'day': 'Sunday', 'open_time': 362340},
           {'close_time': 72000, 'day': 'Monday', 'open_time': 396